In [134]:
import copy

import numpy as np

def parse_board(board):
    return np.array([[int(c) if c.isdigit() else 0 for c in row.strip() if c != " "] for row in board.strip().split('\n')])

def print_board(board):
    for i, row in enumerate(board):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - -")
        for j, val in enumerate(row):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            if j == 8:
                print(val)
            else:
                print(str(val) + " ", end="")
    print()

def board_is_solved(board):

    for num in range(1, 10):
        # Check rows
        for i in range(9):
            count = 0
            for j in range(9): #cycle through columns of a fixed row
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False
                    
        # check columns
        for j in range(9):
            count = 0
            for i in range(9): #cycle through rows of a fixed column
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False

    # Check boxes
        for box_x in range(1, 3):
            for box_y in range(1, 3):
                count = 0
                for i in range(box_y * 3, box_y * 3 + 3):
                    for j in range(box_x * 3, box_x * 3 + 3):
                        if board[i][j] == num:
                            count += 1
                        if(count > 1): #if a number appears more than one time in a box 3x3
                            return False
                if count < 1: #if a number doesn't appear in a box
                    return False
    return True


def board_is_full(board):
    
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return False
    return True

In [135]:
import random

# Genetic Algorithm parameters
POPULATION_SIZE = 50
MUTATION_RATE = 0.3
MAX_GENERATIONS = 10000

def coords_already_full(board):
    
    list = []
    
    for i in range(9):
        for j in range(9):
            if board[i][j] != 0:
                list.append((i,j))
    
    return list
    
def fitness(board):
    fitness_score = 0
    
    for j in range(9):
        seen = {}
        
        for i in range(9): # Check each cell in the column
            if board[i][j] in seen:
                seen[board[i][j]] += 1
            else:
                seen[board[i][j]] = 1
        for key in seen: # Subtract fitness for repeated numbers
            fitness_score -= (seen[key] - 1)
    
    for m in range(3): # For each 3x3 square
        for n in range(3):
            seen = {}
            for i in range(3 * n, 3 * (n + 1)):  # Check cells in 3x3 square
                for j in range(3 * m, 3 * (m + 1)):
                    if board[i][j] in seen:
                        seen[board[i][j]] += 1
                    else:
                        seen[board[i][j]] = 1
            for key in seen: # Subtract fitness for repeated numbers
                fitness_score -= (seen[key] - 1)
            ## check collision in row
            #for j2 in range(9):
            #    # if the element found (different from the starting element) has equal value it's a collision
            #    if board[i, j2] == board[i][j] and (i,j2) != (i,j): 
            #        fitness_score -= 1

            ## check collision in column
            #for i2 in range(9):
            #    if board[i2, j] == board[i][j] and (i2,j) != (i,j):
            #        fitness_score -= 1
                    
            ##check collision in box 3x3
            #box_x = j // 3
            #box_y = i // 3
            #for i2 in range(box_y*3, box_y*3 + 3):
            #    for j2 in range(box_x*3, box_x*3 + 3):
            #        if board[i2, j2] == board[i][j] and (i2,j) != (i,j):
            #            fitness_score -= 1
    
    return fitness_score

def mutate(board, already_filled):
    
    row = random.randint(0, 8)
    
    #while True:
    #    pos1 = random.randint(0, 8)
    #    pos2 = random.randint(0, 8)
    #    if not (row,pos1) in already_filled and not (row,pos2) in already_filled and pos1 != pos2:
    #        break
    #
    #new_board = copy.deepcopy(board)
    #temp = new_board[row][pos1]
    #new_board[row][pos1] = new_board[row][pos2]
    #new_board[row][pos2] = temp
    
    # with this shuffle it is rare to find errors in boxes
    new_board = copy.deepcopy(board)
    
    gene = list()
    # create a list for every row with the numbers from 1 to 9, removing the numbers already present in the row
    for j in range(9):
        if not (row, j) in already_filled:
            gene.append(board[row][j])
    random.shuffle(gene)
    
    for j in range(9):
        if not (row,j) in already_filled:
            new_board[row][j] = gene.pop()
    
    return new_board
    
# todo chiedere a Claude se crossover è meglo farlo verticale, orizzontale o a box, o se selezionare verticale/orizzontale a random
# Vertical crossover of the board
def crossover(parent1, parent2):
    
    child = copy.deepcopy(parent1)

    #for i in range(9):
    #    if random.randint(0, 1) == 0:
    #        child[i] = parent1[i]
    #    else:
    #        child[i] = parent2[i]
    
    crossover_point = random.randint(1, 8)
    for i in range(crossover_point,9):
        child[i] = parent2[i]
        
    return child

# Select parents for crossover using tournament selection
def tournament_selection(population):
    tournament_size = 20
    tournament = random.sample(population, tournament_size)
    return max(tournament, key=lambda x: x[1])

# generate a new board with random numbers (where numbers were not already set)
def generate_board_state(initial_board):
    
    initial = copy.deepcopy(initial_board)
    for i in range(9):
        gene = list()
        # create a list for every row with the numbers from 1 to 9, removing the numbers already present in the row
        for j in range(1,10):
            if not j in initial_board[i]:
                gene.append(j)
        random.shuffle(gene)
        
        for j in range(9):
            if initial_board[i][j] != 0:
                initial[i][j] = initial_board[i][j]
            else:
                initial[i][j] = gene.pop()
    
    return initial

### TEST SECTION

In [136]:
# Example Board
sudoku_board = """
37. 5....6
... 36. .12
... .91 75.
... 154 .7.
..3 .7. 6..
.5. 638 ...
.64 98. ...
59. .26 ...
2.. ..5 .64
"""

sudoku = """
8........
..36.....
.7..9.2..
.5...7...
....45.7.
...1...3.
..1....68
..85...1.
.9....4..
"""

board = parse_board(sudoku_board)

# Generate the initial population
population = [(generate_board_state(board), 0) for _ in range(POPULATION_SIZE)]

# save in a var the coordinates to the cells already filled
already_filled = coords_already_full(board)
# set the best board to the initial board
best_board_state = board

# Main Genetic Algorithm loop
for generation in range(MAX_GENERATIONS):
    # Calculate fitness for each board state
    population = [(board_state, fitness(board_state)) for board_state, _ in population]

    # Check if solution is found
    best_board_state = max(population, key=lambda x: x[1])[0]
    if fitness(best_board_state) == 0:
        print("Solution found in generation", generation)
        break
        
    print("Generazione ", generation)
    print("Fitness: ", max(population, key=lambda x: x[1])[1])
    print_board(best_board_state)
    print()
        
    # Create the next generation
    new_population = []

    # Elitism: Keep the best board state from the previous generation
    new_population.append(max(population, key=lambda x: x[1]))
    
    # Perform selection, crossover, and mutation
    while len(new_population) < POPULATION_SIZE:
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)
        child = crossover(parent1[0], parent2[0])
        if random.random() < MUTATION_RATE:
            child = mutate(child, already_filled)
        new_population.append((child, 0))

    # Update the population
    population = new_population

# Print the best solution
print("Best solution:", best_board_state)
if(board_is_solved(best_board_state)):
    print("Board risolta")

Generazione  0
Fitness:  -35
3 7 4 | 5 8 2 | 9 1 6
7 5 9 | 3 6 8 | 4 1 2
4 8 6 | 2 9 1 | 7 5 3
- - - - - - - - - - - -
9 6 3 | 1 5 4 | 2 7 8
8 1 3 | 4 7 9 | 6 2 5
2 5 7 | 6 3 8 | 9 4 1
- - - - - - - - - - - -
7 6 4 | 9 8 5 | 1 3 2
5 9 4 | 8 2 6 | 3 7 1
2 8 7 | 9 1 5 | 3 6 4


Generazione  1
Fitness:  -29
3 7 1 | 5 2 4 | 9 8 6
5 4 8 | 3 6 7 | 9 1 2
4 8 6 | 2 9 1 | 7 5 3
- - - - - - - - - - - -
9 6 3 | 1 5 4 | 2 7 8
8 1 3 | 4 7 9 | 6 2 5
2 5 7 | 6 3 8 | 9 4 1
- - - - - - - - - - - -
7 6 4 | 9 8 5 | 1 3 2
5 9 4 | 8 2 6 | 3 7 1
2 3 8 | 1 7 5 | 9 6 4


Generazione  2
Fitness:  -26
3 7 1 | 5 2 4 | 9 8 6
9 4 5 | 3 6 7 | 8 1 2
4 8 6 | 2 9 1 | 7 5 3
- - - - - - - - - - - -
9 6 3 | 1 5 4 | 2 7 8
8 1 3 | 4 7 9 | 6 2 5
2 5 7 | 6 3 8 | 9 4 1
- - - - - - - - - - - -
7 6 4 | 9 8 5 | 1 3 2
5 9 4 | 8 2 6 | 3 7 1
2 3 8 | 1 7 5 | 9 6 4


Generazione  3
Fitness:  -24
3 7 1 | 5 2 4 | 9 8 6
8 4 9 | 3 6 7 | 5 1 2
4 8 6 | 2 9 1 | 7 5 3
- - - - - - - - - - - -
9 6 3 | 1 5 4 | 2 7 8
8 1 3 | 4 7 9 | 6 2 5
7 5 4 

KeyboardInterrupt: 

In [ ]:
#print("Example Sudoku:")
#print_board(board)
#
#
#if(solve_board(board)):
#    print("Soluzione: ")
#    print_board(board)
#else:
#    print("Board non risolvibile")

In [ ]:
initial = [[0] * 9] * 9
initial = """
8........
..36.....
.7..9.2..
.5...7...
....45.7.
...1...3.
..1....68
..85...1.
.9....4..
"""
initial = parse_board(initial)
initial[1][1] = 15
print_board(initial)

In [ ]:
for i in range(0,9):
    print(i)